In [ ]:
#!g1.1
!git clone https://github.com/nerel-ds/NEREL

In [ ]:
!unzip NEREL-v1.0.zip -d NEREL

In [528]:
#!g1.1
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [529]:
#!g1.1
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

 60%|██████    | 449/745 [00:01<00:00, 366.60it/s]

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:02<00:00, 355.22it/s]


In [530]:
#!g1.1
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [531]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


# Dataset

In [532]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [533]:
#!g1.1
from transformers import BertForSequenceClassification, BertTokenizer

In [534]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [535]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [536]:
#!g1.1
MAX_LENGTH = 100

In [537]:
#!g1.1
# import nltk
# nltk.download('punkt')
# #!g1.1
# sents = nltk.sent_tokenize(texts[0])
# len(' '.join(sents))

In [554]:
#!g1.1
# class Binary_Tag:
#     def __getitem__(self, x):
#         if (x == 'None'): return 0
#         else: return 1

tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = {s:i for i,s in enumerate(tags)} # string tag --> ids
entity_tags = list(set(e.tag for e in sum(entities,[]))) 

def tok(text1, text2):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1, text2,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]

def order(e1, e2):
    if (len(e1.text) > len(e2.text)):
        return e1, e2
    else:
        return e2, e1


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[e].tag, text_id))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[(e[1], e[0])].tag, text_id))
                else:
                    pre_dataset.append((*order(*e), 'None', text_id))

        data = []
        for e1, e2, tag, text_id in tqdm(pre_dataset):
            
            data.append((reversed_tags[tag], entity_tags.index(e1.tag), entity_tags.index(e2.tag), tag, *tok(e1.text, e2.text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [555]:
#!g1.1
def findPair(i, all_entities, all_relationships_dict):
    j = (i+1)%len(all_entities)
    while True:
        ent1 = all_entities[i]
        ent2 = all_entities[j]
        j = (j+1)%len(all_entities)
        if ((ent1, ent2) not in all_relationships_dict and (ent2, ent1) not in all_relationships_dict):
            return ent1, ent2


class MyDataset2(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        random.seed(2021)
        data = []
        
        for r in tqdm(sum(relationships,[]), desc = 'real tags'):
            e1, e2 = order(r.arg1, r.arg2)
            data.append((reversed_tags[r.tag],  entity_tags.index(e1.tag), entity_tags.index(e2.tag), r.tag, *tok(e1.text, e2.text)))
        
        all_entities = sum(entities, [])
        all_relationships_dict = {(e.arg1, e.arg2):e for e in sum(relationships,[])}
        for _ in tqdm(range(len(sum(relationships,[]))), desc = "'None' tags"):
#             e1, e2 = random.sample(all_entities, 2)
            ent1_id = random.choice(range(len(all_entities)))
            e1, e2 = order(*findPair(ent1_id, all_entities, all_relationships_dict))
            data.append((reversed_tags['None'],  entity_tags.index(e1.tag), entity_tags.index(e2.tag), 'None', *tok(e1.text, e2.text)))

        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [540]:
#!g1.1
train_data_mixed = MyDataset2('NEREL/NEREL-v1.0/train')
train_dl_mixed = DataLoader(train_data_mixed, shuffle=True, batch_size=32)

train_data_normal = MyDataset('NEREL/NEREL-v1.0/train')
train_dl_normal = DataLoader(train_data_normal, shuffle=True, batch_size=32)

dev_data_mixed = MyDataset2('NEREL/NEREL-v1.0/dev')
dev_dl_mixed = DataLoader(dev_data_mixed, batch_size=32)

dev_data_normal = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl_normal = DataLoader(dev_data_normal, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text



incorrect format in: row=По словам очевидцев пешехо
 file=21013_text



# Training

In [577]:
#!g1.1
from transformers import BertTokenizer, BertModel
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            max_length = MAX_LENGTH
            )
        self.dropout = nn.Dropout(0.1)
        self.emb = nn.Embedding(29, 10)
        self.fc = nn.Sequential(
            nn.Linear(768 + 10*2, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, len(tags))
            )
        
    def forward(self, ent1, ent2, input_ids, attention_mask, token_type_ids, labels):
        ent1 = self.emb(ent1)
        ent2 = self.emb(ent2)
        out = self.bert(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)[1]
        out = self.dropout(out)
        out = torch.cat((out, ent1, ent2), dim=1)
        out = self.fc(out)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(out.view(-1, len(tags)), labels.view(-1))
        return loss, out
    
        # requires reconfiguration of hyper parameters
#         criterion = nn.BCEWithLogitsLoss()
#         out = self.fc(out).view(-1)
#         loss = criterion(out, labels)
#         return loss, torch.stack((-torch.sigmoid(out), torch.sigmoid(out)), dim = 1)

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [542]:
#!g1.1
model = MyModel()
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [543]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.argmax(y_pred, axis = 1) == y)*1.0/len(y)

def train(train_dl):
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(tag1, tag2,
                    input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)
        
        loss = outputs[0]
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
    print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

    
from sklearn.metrics import f1_score
def test(dev_dl):
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

#         dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/train'), batch_size = 32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)

            outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

            loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
            
            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
#             pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
#     print(y_true, y_pred)
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}')
    
    y_true_bin = [0 if x != reversed_tags['None'] else 1 for x in y_true]
    y_pred_bin = [0 if x != reversed_tags['None'] else 1 for x in y_pred]
    
    bin_acc = sum(a == b for a,b in zip(y_true_bin, y_pred_bin))/len(y_pred)
    print(f'test accuracy on binary classification = {bin_acc}, F1 = {f1_score(y_true_bin, y_pred_bin)}')

In [512]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
scheduler = ReduceLROnPlateau(optimizer, patience = 3, cooldown = 1, factor = 0.5)

In [513]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 6):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train(train_dl_mixed)
    print('mixed dataset')
    test(dev_dl_mixed)
    print('nested dataset')
    test(dev_dl_normal)

epoch = 1
lr= 5e-05



train loss= 1.2492 
 train accuracy = 0.6523,
mixed dataset



test loss= 0.9488 
 test accuracy = 0.6943
test accuracy on Binary class = 0.751937984496124, F1 = 0.7654532128023455
nested dataset



test loss= 0.6705 
 test accuracy = 0.8040
test accuracy on Binary class = 0.8316679477325134, F1 = 0.859344894026975
epoch = 2
lr= 5e-05



train loss= 0.7031 
 train accuracy = 0.7748,
mixed dataset


...


test loss= 1.0448 
 test accuracy = 0.6945
test accuracy on Binary class = 0.7760981912144703, F1 = 0.7680984878897363
nested dataset



test loss= 0.7182 
 test accuracy = 0.7682
test accuracy on Binary class = 0.8009223674096848, F1 = 0.8248816768086545
epoch = 5
lr= 5e-05



train loss= 0.3191 
 train accuracy = 0.9011,
mixed dataset



test loss= 1.1272 
 test accuracy = 0.6883
test accuracy on Binary class = 0.7698966408268734, F1 = 0.7601346801346801
nested dataset



test loss= 0.7961 
 test accuracy = 0.7769
test accuracy on Binary class = 0.8124519600307456, F1 = 0.834912043301759


In [515]:
#!g1.1
for epoch in range(1, 21):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train(train_dl_normal)
    print('nested dataset')
    test(dev_dl_normal)

epoch = 1
lr= 5e-05



train loss= 0.2167 
 train accuracy = 0.9350,
nested dataset



test loss= 0.6938 
 test accuracy = 0.8185
test accuracy on Binary class = 0.846272098385857, F1 = 0.8732572877059569
epoch = 2
lr= 5e-05



train loss= 0.1724 
 train accuracy = 0.9462,
nested dataset

test loss= 0.7647 
 test accuracy = 0.8369
test accuracy on Binary class = 0.8554957724827056, F1 = 0.8882282996432819
epoch = 3
lr= 5e-05



train loss= 0.1591 
 train accuracy = 0.9531,
nested dataset


...


test loss= 1.1826 
 test accuracy = 0.8227
test accuracy on Binary class = 0.857033051498847, F1 = 0.8819796954314723
epoch = 18
lr= 5e-05



train loss= 0.0731 
 train accuracy = 0.9799,
nested dataset



test loss= 1.1260 
 test accuracy = 0.8246
test accuracy on Binary class = 0.8485780169100692, F1 = 0.8769519050593378
epoch = 19
lr= 2.5e-05



train loss= 0.0499 
 train accuracy = 0.9847,
nested dataset

test loss= 1.2744 
 test accuracy = 0.8227
test accuracy on Binary class = 0.8508839354342813, F1 = 0.8805418719211823
epoch = 20
lr= 2.5e-05



train loss= 0.0380 
 train accuracy = 0.9877,
nested dataset

test loss= 1.2891 
 test accuracy = 0.8262
test accuracy on Binary class = 0.8508839354342813, F1 = 0.8819951338199513


In [527]:
#!g1.1
torch.save(model.state_dict(), 'models/Set2.5_BERT')

In [578]:
#!g1.1
model = MyModel()
model.load_state_dict(torch.load('models/Set2.5_BERT'))
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [587]:
#!g1.1

def to_bin(x):
    return 0 if x != reversed_tags['None'] else 1

with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/dev'), batch_size = 1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []
#     tags = []
    
    mis_tags = []
    rel_tags = []
    texts = []
    found_expected = []

    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)

        outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

        loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
#         tags += [(entity_tags[tag1], entity_tags[tag2])]
        
        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        if (to_bin(torch.argmax(outputs[1], axis = 1)) != to_bin(labels)):
            print('expected:', tags[labels.detach().cpu().item()])
#             print('found:', torch.argmax(outputs[1], axis = 1).detach().cpu().item(), '(', outputs[1].detach().cpu().numpy() ,')')
            print('found:', tags[torch.argmax(outputs[1], axis = 1).detach().cpu().item()])
            print(f'{entity_tags[tag1]} ---> {entity_tags[tag2]} : {true_tag[0]}')
            print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))
            
            
            mis_tags.append((entity_tags[tag1], entity_tags[tag2]))
            texts.append(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))
            found_expected.append((tags[torch.argmax(outputs[1], axis = 1).detach().cpu().item()], tags[labels.detach().cpu().item()]))
            rel_tags.append(true_tag[0])
            print()
            
print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}')


expected: None
found: PRICE_OF
ORGANIZATION ---> PERSON : None
[CLS] Строгановское высшее художественное училище [SEP] Строгановское [SEP]                                                                                          

expected: ORIGINS_FROM
found: None
PROFESSION ---> COUNTRY : ORIGINS_FROM
[CLS] сотрудников исправительных учреждений Италии [SEP] Италии [SEP]                                                                                            

expected: None
found: SUBORDINATE_OF
PROFESSION ---> COUNTRY : None
[CLS] Премьер - министр Великобритании [SEP] Великобритании [SEP]                                                                                            

expected: WORKPLACE
found: None
PROFESSION ---> ORGANIZATION : WORKPLACE
[CLS] Гендиректор ООО УК « Металлоинвест [SEP] Металлоинвест [SEP]                                                                                         

expected: None
found: DATE_OF_BIRTH
EVENT ---> DISEASE : None
[CLS] пандеми

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [590]:
#!g1.1
from collections import Counter
print('\n'.join([f'{a} ---> {b} : {n}' for (a,b),n in Counter(mis_tags).most_common()]))

PROFESSION ---> ORGANIZATION : 25
ORGANIZATION ---> ORGANIZATION : 14
ORGANIZATION ---> STATE_OR_PROVINCE : 10
EVENT ---> LOCATION : 10
ORGANIZATION ---> COUNTRY : 9
EVENT ---> PERSON : 8
PROFESSION ---> COUNTRY : 7
FACILITY ---> PERSON : 7
EVENT ---> COUNTRY : 6
WORK_OF_ART ---> PERSON : 6
PROFESSION ---> STATE_OR_PROVINCE : 6
ORGANIZATION ---> PERSON : 5
LAW ---> LAW : 5
EVENT ---> EVENT : 5
DISEASE ---> DISEASE : 5
PROFESSION ---> PROFESSION : 5
EVENT ---> DISEASE : 4
ORGANIZATION ---> PROFESSION : 4
EVENT ---> STATE_OR_PROVINCE : 4
LAW ---> PERSON : 3
PENALTY ---> MONEY : 3
ORGANIZATION ---> CITY : 3
ORGANIZATION ---> LOCATION : 2
LAW ---> COUNTRY : 2
PROFESSION ---> LOCATION : 2
ORGANIZATION ---> DISTRICT : 2
EVENT ---> WORK_OF_ART : 2
EVENT ---> FACILITY : 2
WORK_OF_ART ---> COUNTRY : 2
ORGANIZATION ---> RELIGION : 2
ORGANIZATION ---> NATIONALITY : 2
LAW ---> ORGANIZATION : 1
EVENT ---> ORGANIZATION : 1
LAW ---> STATE_OR_PROVINCE : 1
PROFESSION ---> RELIGION : 1
CRIME ---> LAW : 

In [591]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('PROFESSION', 'ORGANIZATION'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = None
expected = WORKPLACE
PROFESSION ---> ORGANIZATION : WORKPLACE
[CLS] Гендиректор ООО УК « Металлоинвест [SEP] Металлоинвест [SEP]                                                                                         
found    = SUBORDINATE_OF
expected = None
PROFESSION ---> ORGANIZATION : None
[CLS] руководителя Приволжского межрегионального координационного совета « Единой России » [SEP] Приволжского межрегионального координационного совета « Единой России » [SEP]                                                                              
found    = None
expected = WORKPLACE
PROFESSION ---> ORGANIZATION : WORKPLACE
[CLS] художественным директором галереи « Манеж [SEP] Манеж [SEP]                                                                                           
found    = SUBORDINATE_OF
expected = None
PROFESSION ---> ORGANIZATION : None
[CLS] Вокалист британской группы The Prodigy [SEP] The Prodigy [SEP]                                                      

In [592]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('ORGANIZATION', 'COUNTRY'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = None
expected = HEADQUARTERED_IN
ORGANIZATION ---> COUNTRY : HEADQUARTERED_IN
[CLS] Российской Федерации Северной ходьбы [SEP] Российской [SEP]                                                                                            
found    = PRICE_OF
expected = None
ORGANIZATION ---> COUNTRY : None
[CLS] Суд в Италии [SEP] Италии [SEP]                                                                                             
found    = PRICE_OF
expected = None
ORGANIZATION ---> COUNTRY : None
[CLS] Всекитайского собрания народных представителей [SEP] Всекитайского [SEP]                                                                                      
found    = PRICE_OF
expected = None
ORGANIZATION ---> COUNTRY : None
[CLS] Национальной системы здравоохранения Великобритании [SEP] Великобритании [SEP]                                                                                            
found    = None
expected = HEADQUARTERED_IN
ORGANIZATION ---> COUNTRY : H

In [593]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('EVENT', 'LOCATION'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = None
expected = TAKES_PLACE_IN
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = None
expected = TAKES_PLACE_IN
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] Встреча на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = None
expected = TAKES_PLACE_IN
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] Встреча на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = None
expected = TAKES_PLACE_IN
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = None
expected = TAKES_PLACE_IN
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                             

In [594]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('EVENT', 'PERSON'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = None
expected = ORIGINS_FROM
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Международного конкурса имени Петра Чайковского [SEP] Петра Чайковского [SEP]                                                                                          
found    = None
expected = ORIGINS_FROM
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Конкурса имени Чайковского [SEP] Чайковского [SEP]                                                                                            
found    = None
expected = PARTICIPANT_IN
EVENT ---> PERSON : PARTICIPANT_IN
[CLS] Марша Голунова [SEP] Голунова [SEP]                                                                                            
found    = None
expected = ORIGINS_FROM
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Шукшинским дням на Алтае [SEP] Шукшинским [SEP]                                                                                        
found    = DATE_OF_CREATION
expected = None
EVENT ---> PERSON : None
[CLS] Шукшинский кинофестиваль [SEP]